Reproducing the double descent with Gaussian matrices as a sanity check

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from VAMP_toolbox_clean import *
from math import isnan

In [2]:
# Gaussian sanity check, all i.i.d, generalization error as a function of the aspect ratio

n_exp = 50
asp_vec = np.linspace(0.5,3,n_exp) # aspec ratios 
E_vec = np.zeros(n_exp)
rho = 0.3  # sparsity parameter of the teacher
delta0 = 0.01
n,d = 200,100 # low dimension, still gives very good agreement with asymptotics
n_avg = 20 # choose the averaging number 

In [ ]:
for i in range(n_exp):
    print((i+1)/n_exp*100,'%')
    for k in range(n_avg):  # 20 times averaging, solving the minimization problem with sklearn elastic net
        
            asp = asp_vec[i]
            n = int(asp*d)
            x0 = gauss_bernoulli(rho,d)
            F_train = np.random.normal(0,np.sqrt(1/n),(n,d))
            w = np.random.normal(0,np.sqrt(delta0),n)
            y_train = F_train@x0+w
            clf = linear_model.ElasticNet(alpha=0.001/n,l1_ratio=1,max_iter = 100000,tol = 0.00001)
            clf.fit(F_train,y_train)
            x_train = clf.coef_
            E_vec[i] = E_vec[i]+np.mean((x0-x_train)**2)
            
    E_vec[i] = 1/n_avg*E_vec[i]

In [ ]:
# Gaussian sanity check, all i.i.d - Theory using SE asymptotics/Replica prediction

n_th = 51
asp_vec_th = np.linspace(0.5,3,n_th)
E_vec_th = np.zeros(n_th)
reg = 0.001
niter = 100
my_eps = 0

for i in range(n_th):
    print((i+1)/n_th*100,'%')
    asp = asp_vec_th[i]
    damp_se = 1
        
    alpha1_vec,alpha2_vec,eta1_vec,eta2_vec,gamma1_vec,gamma2_vec,E1_vec,E2_vec,tau1_vec,tau2_vec = VAMP_SE_l1_unif(asp,rho,delta0,reg,niter,damp_se,my_eps,0)
    
    V = 1/2*(1/eta1_vec[-1]+1/eta2_vec[-1])
    A1 = alpha1_vec[-1]/V
    A2 = alpha2_vec[-1]/V
    tau1 = tau1_vec[-1]
    tau2 = tau2_vec[-1]
    E = 1/2*(E1_vec[-1]+E2_vec[-1])
        
    while isnan(E) == True:
        print('restarting iteration')
        alpha1_vec,alpha2_vec,eta1_vec,eta2_vec,gamma1_vec,gamma2_vec,E1_vec,E2_vec,tau1_vec,tau2_vec = VAMP_SE_l1_unif(asp,rho,delta0,reg,niter,damp_se,my_eps,0)
        
        V = 1/2*(1/eta1_vec[-1]+1/eta2_vec[-1])
        A1 = alpha1_vec[-1]/V
        A2 = alpha2_vec[-1]/V
        tau1 = tau1_vec[-1]
        tau2 = tau2_vec[-1]
        E = 1/2*(E1_vec[-1]+E2_vec[-1])
        
        
    E_vec_th[i] = (E1_vec[-1]+E2_vec[-1])/2

In [ ]:
# Plot similar to all other papers on Gaussian double descent

plt.plot(asp_vec,E_vec,'.',label = 'Experiment')
plt.plot(asp_vec_th,E_vec_th, label = 'Theory')
plt.legend()
plt.title('Gaussian sanity check d=100')
plt.xlabel('aspect_ratio')
plt.ylabel('risk')
plt.savefig('sanity_check_Gaussian', dpi=500, quality = 95)